In [9]:
from sklearn.metrics import r2_score, mean_squared_error
import joblib  # save model
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from numpy import absolute
from numpy import std
from numpy import mean
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from sklearn.svm import LinearSVR
from sklearn.datasets import make_regression
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV

In [10]:

file = r'dataset\features\EURUSD.csv'
df = pd.read_csv(file)
df.set_index('date', inplace=True, drop=True)

################################################
features = df[['open',
 'high',
 'low',
 'close',
 'WPC',
 'HA_open',
 'HA_high',
 'HA_low',
 'HA_close',
 'EMA_4',
 'EMA_8',
 'EMA_16',
 'EMA_32',
 'bb_bbm_4',
 'bb_bbh_4',
 'bb_bbl_4',
 'bb_bbm_8',
 'bb_bbh_8',
 'bb_bbl_8',
 'bb_bbm_16',
 'bb_bbh_16',
 'bb_bbl_16',
 'bb_bbm_32',
 'bb_bbh_32',
 'bb_bbl_32']].copy()
targets = df[['open_24', 'close_24']].copy()

################################################
targets = targets*10000
################################################

In [11]:
sc_X = StandardScaler()
sc_y = StandardScaler()
x = sc_X.fit_transform(features.values)
y = sc_y.fit_transform(targets.values)

input_train, input_test, output_train, output_test = train_test_split(
    x, y, test_size=0.05)
print("train shape : {:.0f}".format(
    input_train.shape[0]/24), "days || test shape : {:.0f}".format(input_test.shape[0]/24), "days")

train shape : 1405 days || test shape : 74 days


In [22]:
param = {
    'estimator__kernel' : ('poly','rbf','sigmoid'),
    'estimator__C' : np.arange(1,52,10),
    'estimator__degree' : np.arange(3,8,1),
    'estimator__coef0' :  np.arange(0.001,10,0.5),
    'estimator__gamma' : ('auto','scale',0.01,0.1,1,10),
    'estimator__epsilon' : (0.0001,0.001,0.01,0.1,1),
    'estimator__verbose' : [2]
}
param

{'estimator__kernel': ('poly', 'rbf', 'sigmoid'),
 'estimator__C': array([ 1, 11, 21, 31, 41, 51]),
 'estimator__degree': array([3, 4, 5, 6, 7]),
 'estimator__coef0': array([1.000e-03, 5.010e-01, 1.001e+00, 1.501e+00, 2.001e+00, 2.501e+00,
        3.001e+00, 3.501e+00, 4.001e+00, 4.501e+00, 5.001e+00, 5.501e+00,
        6.001e+00, 6.501e+00, 7.001e+00, 7.501e+00, 8.001e+00, 8.501e+00,
        9.001e+00, 9.501e+00]),
 'estimator__gamma': ('auto', 'scale', 0.01, 0.1, 1, 10),
 'estimator__epsilon': (0.0001, 0.001, 0.01, 0.1, 1),
 'estimator__verbose': [2]}

In [23]:
model = SVR()
Multi_SVR = MultiOutputRegressor(model)
Grids = GridSearchCV(Multi_SVR,param)
Multi_SVR.get_params().keys()

dict_keys(['estimator__C', 'estimator__cache_size', 'estimator__coef0', 'estimator__degree', 'estimator__epsilon', 'estimator__gamma', 'estimator__kernel', 'estimator__max_iter', 'estimator__shrinking', 'estimator__tol', 'estimator__verbose', 'estimator', 'n_jobs'])

In [14]:
Grids.fit(input_train,output_train)

ValueError: Invalid parameter C for estimator MultiOutputRegressor(estimator=SVR()). Check the list of available parameters with `estimator.get_params().keys()`.

In [ ]:
print("Best score : ",Grids.best_score_)
print("Best param : ",Grids.best_params_)

In [ ]:
Grids.cv_results_